In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import init_utils
import reg_utils
import gc_utils
import testCases
%matplotlib inline
plt.rcParams['figure.figsize'] = (7, 4)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
train_X, train_Y, test_X, test_Y = init_utils.load_dataset(is_plot = True)

三种初始化参数对比

In [ ]:

def initialize_parameters_zeros(layers_dims):
    # layers_dims包括输入层
    parameters = {}
    num_layers = len(layers_dims)
    for i in range(1, num_layers):
        parameters["W" + str(i)] = np.zeros((layers_dims[i], layers_dims[i - 1]))
        parameters["b" + str(i)] = np.zeros((layers_dims[i], 1))
        assert(parameters["W" + str(i)].shape == (layers_dims[i], layers_dims[i - 1]))
    return parameters

def initialize_parameters_random(layers_dims):
    np.random.seed(1)
    parameters = {}
    num_layers = len(layers_dims)
    for i in range(1, num_layers):
        parameters["W" + str(i)] = np.random.randn(layers_dims[i], layers_dims[i - 1]) * 10
        parameters["b" + str(i)] = np.zeros((layers_dims[i], 1))
        assert(parameters["W" + str(i)].shape == (layers_dims[i], layers_dims[i - 1]))
    return parameters

def initialize_parameters_he(layers_dims):
    parameters = {}
    np.random.seed(1)
    num_layers = len(layers_dims)
    for i in range(1, num_layers):
        parameters["W" + str(i)] = np.random.randn(layers_dims[i], layers_dims[i - 1]) * np.sqrt(2 / layers_dims[i - 1])
        parameters["b" + str(i)] = np.zeros((layers_dims[i], 1))
        assert(parameters["W" + str(i)].shape == (layers_dims[i], layers_dims[i - 1]))
    return parameters

In [ ]:
parameters = initialize_parameters_zeros([3,2,1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

parameters = initialize_parameters_random([3, 2, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

parameters = initialize_parameters_he([2, 4, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))


In [ ]:
def model(X, Y, learning_rate = 0.01, num_iterations = 15000, print_cost = True, initialization = "he", is_polt = True):
    costs = []
    layers_dims = [X.shape[0], 10, 5, 1]
    
    if initialization == "zeros":
        parameters = initialize_parameters_zeros(layers_dims)
    elif initialization == "random":
        parameters = initialize_parameters_random(layers_dims)
    elif initialization == "he":
        parameters = initialize_parameters_he(layers_dims)
    else:
        print('initializing occurs error and program exits')
        exit
    
    for i in range(num_iterations):
        a3, cache = init_utils.forward_propagation(X, parameters)
        cost = init_utils.compute_loss(a3, Y)
        grads = init_utils.backward_propagation(X, Y, cache)
        parameters = init_utils.update_parameters(parameters, grads, learning_rate)
        if i % 1000 == 0:
            costs.append(cost)
            if print_cost:
                print("Iterations: {}\t|cost:{}".format(i, cost))
    if is_polt:
        plt.plot(costs)
        plt.ylabel('cost')
        plt.xlabel('iterations')
        plt.title("learning rate :" + str(learning_rate))
        
    return parameters

In [ ]:
# 初始化为0
parameters = model(train_X, train_Y, initialization = "zeros")
print ("训练集:")
predictions_train = init_utils.predict(train_X, train_Y, parameters)
print ("测试集:")
predictions_test = init_utils.predict(test_X, test_Y, parameters)

In [ ]:
#随机初始化
parameters = model(train_X, train_Y, initialization = "random",is_polt=True)
print("训练集：")
predictions_train = init_utils.predict(train_X, train_Y, parameters)
print("测试集：")
predictions_test = init_utils.predict(test_X, test_Y, parameters)

print(predictions_train)
print(predictions_test)

#随机初始化边界图
plt.title("Model with large random initialization")
axes = plt.gca()
axes.set_xlim([-1.5, 1.5])
axes.set_ylim([-1.5, 1.5])
init_utils.plot_decision_boundary(lambda x: init_utils.predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
# 抑梯度异常初始化
parameters = model(train_X, train_Y, initialization = "he",is_polt=True)
print("训练集:")
predictions_train = init_utils.predict(train_X, train_Y, parameters)
print("测试集:")
init_utils.predictions_test = init_utils.predict(test_X, test_Y, parameters)
# 边界图
plt.title("Model with He initialization")
axes = plt.gca()
axes.set_xlim([-1.5, 1.5])
axes.set_ylim([-1.5, 1.5])
init_utils.plot_decision_boundary(lambda x: init_utils.predict_dec(parameters, x.T), train_X, train_Y)


正则化模型

In [ ]:
train_X, train_Y, test_X, test_Y = reg_utils.load_2D_dataset(is_plot = True)

In [ ]:
def forward_propagation_with_dropout(X, parameters, keep_prob = 0.5):  # dropout的前向传播
    np.random.seed(1)
    w1 = parameters["W1"]
    b1 = parameters["b1"]
    w2 = parameters["W2"]
    b2 = parameters["b2"]
    w3 = parameters["W3"]
    b3 = parameters["b3"]
    
  
    Z1 = np.dot(w1, X) + b1
    A1 = reg_utils.relu(Z1)
    D1 = np.random.rand(A1.shape[0], A1.shape[1])
    D1 = D1 < keep_prob
    A1 = A1 * D1
    A1 = A1 /  keep_prob
    
    Z2 = np.dot(w2, A1) + b2
    A2 = reg_utils.relu(Z2)
    D2 = np.random.rand(A2.shape[0], A2.shape[1])
    D2 = D2 < keep_prob
    A2 = A2 * D2
    A2 = A2 / keep_prob
    
    Z3 = np.dot(w3, A2) + b3
    A3 = reg_utils.sigmoid(Z3)
    
    cache = (Z1, D1, A1, w1, b1, Z2, D2, A2, w2, b2, Z3, A3, w3, b3)
    return A3, cache

def compute_cost_with_regularization(A_output, Y, parameters, lambd):  # L2正则化的loss
    m = Y.shape[1]
    w1 = parameters["W1"]
    w2 = parameters["W2"]
    w3 = parameters["W3"]
    cross_entropy_cost = reg_utils.compute_cost(A_output, Y)
    L2_regularization_cost = (lambd / (2 * m)) * (np.sum(np.square(w1)) + np.sum(np.square(w2)) + np.sum(np.square(w2)))
    loss = cross_entropy_cost + L2_regularization_cost
    return loss

def  backward_propagation_with_regularization(X, Y,cache, lambd): # L2正则化的反向传播,参数w和b保存在cache中
    # 返回一次反向传播的梯度
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    m = X.shape[1]
    dZ3 = A3 - Y #起步是dz，交叉熵损失对Z3求导之后，经计算就是A3 - Y
    dW3 = (1 / m) * np.dot(dZ3, A2.T) + (lambd / m) * W3
    db3 = (1 / m) * np.sum(dZ3, axis = 1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0 )) # A2 = relu(Z2),所以，当Z2>0时，导数是1
    dW2 = (1 / m) * np.dot(dZ2, A1.T) + (lambd / m) * W2
    db2 = (1 / m) * np.sum(dZ2, axis = 1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = (1 / m) * np.dot(dZ1, X.T) + (lambd / m) * W1
    db1 = (1 / m) * np.sum(dZ1, axis = 1, keepdims = True)
    
    grads = {"dZ3":dZ3, "dW3":dW3, "db3":db3,
             "dA2":dA2, "dZ2":dZ2, "dW2":dW2, "db2":db2,
             "dA1":dA1, "dZ1":dZ1, "dW1":dW1, "db1":db1
            }
    return grads
def backward_propagation_with_dropout(X, Y, cache, keep_prob):
    # 先正常前向传播，再用cache的D进行和前向传播同样的dropout处理
    (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache
    m = X.shape[1]
    dz3 = A3 - Y #起步，由于loss的特殊性，计算后dz3 = A3 - Y
    dw3 = (1 / m) * np.dot(dz3, A2.T)
    db3 = (1 / m) * np.sum(dz3, axis = 1, keepdims = True)
    
    da2 = np.dot(W3.T, dz3)
    da2 = da2 * D2
    #da2 = da2 / (1 - keep_prob)
    da2 = da2 / keep_prob
    dz2 = np.multiply(da2, np.int64(A2 > 0)) 
    dw2 = (1 / m) * np.dot(dz2, A1.T)
    db2 = (1 / m) * np.sum(dz2, axis = 1, keepdims = True)
    
    da1 = np.dot(W2.T, dz2)
    da1 = da1 * D1
    #da1 = da1 / (1 - keep_prob)
    da1 = da1 / keep_prob
    dz1 = np.multiply(da1, np.int64(A1 > 0))
    dw1 = (1 / m) * np.dot(dz1, X.T)
    db1 = (1 / m) * np.sum(dz1, axis = 1, keepdims = True)
    
    grads = {"dZ3":dz3, "dW3":dw3, "db3":db3,
             "dA2":da2, "dZ2":dz2, "dW2":dw2, "db2":db2,
             "dA1":da1, "dZ1":dz1, "dW1":dw1, "db1":db1    
            }   
    return grads

In [ ]:
def model(X, Y, learning_rate = 0.3, num_iterations = 30000, print_cost = True, is_plot=True, lambd = 0, keep_prob = 1):
    costs = []
    layers_dims = [X.shape[0], 20, 3, 1]
    parameters = reg_utils.initialize_parameters(layers_dims)
    for i in range(num_iterations):
        #前向传播
        if keep_prob == 1:
            A_output, cache = reg_utils.forward_propagation(X, parameters)
        elif keep_prob < 1:
            A_output, cache = forward_propagation_with_dropout(X, parameters, keep_prob)  # dropout
        else :
            print("keep_prob error")
            exit
            
        # 计算loss   
        if lambd == 0:
            cost = reg_utils.compute_cost(A_output, Y)
        else:
            cost = compute_cost_with_regularization(A_output, Y, parameters, lambd)
            
        # 反向传播
        if (lambd == 0 and keep_prob == 1):
            #不正则化，也不使用dropout
            grads = reg_utils.backward_propagation(X, Y, cache)
        elif lambd != 0:
            # L2正则化
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            # dropout正则化
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)
        
        parameters = reg_utils.update_parameters(parameters, grads, learning_rate)
        
        if i % 1000 == 0:
            costs.append(cost)
            if print_cost:
                print("iteration:{}\t | cost: {}".format(i, cost))
                
    if is_plot:
        plt.plot(costs)
        plt.ylabel("cost")
        plt.xlabel("iterations")
        plt.title("learning rate :" + str(learning_rate))
        plt.show()
    
    return parameters




In [ ]:
# # 测试不使用正则化
# parameters = model(train_X, train_Y,is_plot=True)
# print("训练集:")
# predictions_train = reg_utils.predict(train_X, train_Y, parameters)
# print("测试集:")
# predictions_test = reg_utils.predict(test_X, test_Y, parameters)
# # 边界图
# plt.title("Model without regularization")
# axes = plt.gca()
# axes.set_xlim([-0.75,0.40])
# axes.set_ylim([-0.75,0.65])
# reg_utils.plot_decision_boundary(lambda x: reg_utils.predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
# # 测试使用正则化
# parameters = model(train_X, train_Y, lambd=0.7,is_plot=True)
# print("使用正则化，训练集:")
# predictions_train = reg_utils.predict(train_X, train_Y, parameters)
# print("使用正则化，测试集:")
# predictions_test = reg_utils.predict(test_X, test_Y, parameters)
# # 画边界线
# plt.title("Model with L2-regularization")
# axes = plt.gca()
# axes.set_xlim([-0.75,0.40])
# axes.set_ylim([-0.75,0.65])
# reg_utils.plot_decision_boundary(lambda x: reg_utils.predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
# #测试使用dropout
# parameters = model(train_X, train_Y, learning_rate=0.3, is_plot=True, keep_prob=0.86)

# print("使用随机删除节点，训练集:")
# predictions_train = reg_utils.predict(train_X, train_Y, parameters)
# print("使用随机删除节点，测试集:")
# reg_utils.predictions_test = reg_utils.predict(test_X, test_Y, parameters)
# # 画出决策边界
# plt.title("Model with dropout")
# axes = plt.gca()
# axes.set_xlim([-0.75, 0.40])
# axes.set_ylim([-0.75, 0.65])
# reg_utils.plot_decision_boundary(lambda x: reg_utils.predict_dec(parameters, x.T), train_X, train_Y)

梯度检查

In [ ]:
def forward_propagation_n(X, Y, parameters):
    # 前向传播并计算loss
    m = X.shape[1]
    w1 = parameters["W1"]
    b1 = parameters["b1"]
    w2 = parameters["W2"]
    b2 = parameters["b2"]
    w3 = parameters["W3"]
    b3 = parameters["b3"]
    
    z1 = np.dot(w1, X) + b1
    a1 = reg_utils.relu(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = reg_utils.relu(z2)
    z3 = np.dot(w3, a2) + b3
    a3 = reg_utils.sigmoid(z3)
    
    cost = -(1 / m) *np.sum( Y * np.log(a3) + (1 - Y) * np.log(1 - a3))
    cache = (z1, a1, w1, b1, z2, a2, w2, b2, z3, a3, w3, b3)
    return cost,cache

def backward_propagation_n(X, Y, cache):
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    m = X.shape[1]
    dZ3 = A3 - Y
    dW3 = (1 / m) * np.dot(dZ3, A2.T)
    db3 = (1 / m) * np.sum(dZ3, axis = 1, keepdims = True)
    
    dA2 = np.dot(W3.T,dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis = 1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis = 1, keepdims = True)
    
    grads = {"dZ3":dZ3, "dW3":dW3, "db3":db3,
             "dA2":dA2, "dZ2":dZ2, "dW2":dW2, "db2":db2,
             "dA1":dA1, "dZ1":dZ1, "dW1":dW1, "db1":db1        
    }
    
    
    return grads

def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    # 设置变量
    parameters_values, _ = gc_utils.dictionary_to_vector(parameters)  # 将参数变成向量
    grad = gc_utils.gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # 计算gradaprox
    for i in range(num_parameters):
        thetaplus = np.copy(parameters_values)
        thetaplus[i][0] = thetaplus[i][0] + epsilon
        J_plus[i], cache = forward_propagation_n(X, Y, gc_utils.vector_to_dictionary(thetaplus))  # 只改变了第i个参数，求出J(theta[i] + epsilon)
        
        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] = thetaminus[i][0] - epsilon
        J_minus[i],_ = forward_propagation_n(X, Y, gc_utils.vector_to_dictionary(thetaminus))
        
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
        
    
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator
    
    if difference < 1e-7:
        print("gradients normal, difference: {}".format(difference))
    else :
        print("gradients error, difference: {}".format(difference))
        
    return difference
    
    

In [ ]:
X, Y, parameters = testCases.gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)
